In [1]:
#Make userDict
import pandas as pd # import pandas library
import time
import plotly.express as px

df = pd.read_csv('Reviews.csv', low_memory=False)# read the csv file into a pandas dataframe object
print(df.columns)
#len(pd.unique(df["UserId"].values.ravel()))

userDict = {}

for index, row in df.iterrows():
    if row["UserId"] in userDict:
        userDict[row["UserId"]].append(row)
    else:
        userDict[row["UserId"]] = [row]
        
len(userDict)


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')


256059

In [2]:
#10-05: Make simplifiedReviews file
import csv
result = open('simplifiedReviews.csv', 'w')
count = 0 #Count variable bypasses the first line of the file, which just contains the headers.
resultLength = 0
with open('Reviews.csv') as totalFile:
    lines = totalFile.readlines()
    wr = csv.writer(result)
    for review in lines:
        if count != 0 :
            review = review.split(',')
            user = review[2]
            #print(user)
            #print(count)
            if len(userDict[user]) >= 60 and len(userDict[user]) < 80: #whatever this actually is for you :)
                wr.writerow(review) 
                resultLength +=1
        count+=1

totalFile.close()
result.close()

file = open("simplifiedReviews.csv")
lines= len(list(csv.reader(file)))
file.close()
print(lines, resultLength) # checks that the number of lines that should be added = number of lines in the csv
print(count) #make sure it read all 568455 reviews

5453 5453
568455


In [3]:
#10-05: Make simplified mainDict
from datetime import datetime
from collections import defaultdict
from operator import itemgetter

mainDictionary = defaultdict(lambda: defaultdict(list))

freqDict = defaultdict(lambda: defaultdict(int))

with open('simplifiedReviews.csv') as reviewFile:
    reviews = reviewFile.readlines()
    for r in reviews:
        if reviews[0] == r:
            continue
        r = r.split(',')
        if len(r) > 1:
            userId = r[2]
            productId = r[1]
            hNum = r[4]
            hDen = r[5]
            score = r[6]
            time = r[7]
            length = len(r[8])
               
            isFound = False
            betterDate = datetime.utcfromtimestamp(float(time))
            try:
                infoList = [int(hNum), int(hDen), int(score), productId]
                
                for date in mainDictionary[userId].values():
                    for review in date:
                        if review[3] == productId:
                            isFound = True
                            break
                if not isFound:
                    mainDictionary[userId][(betterDate.month, betterDate.year)].append(infoList) #instead of append, Molly had ... betterDate.year)] = infoList, but I don't think that's right (??)
                    freqDict[userId][(betterDate.month, betterDate.year)] += 1

            except ValueError:
                continue
                
for user in mainDictionary.keys() :
    for d in mainDictionary[user].keys() :
        mainDictionary[user][d] = sorted(mainDictionary[user][d], key = itemgetter(0), reverse=True)
 
        
print("done")
#print(mainDictionary)

#this calculates the number of reviews in mainDictionary
numberOfReviews = 0
for user in mainDictionary.keys():
    for date in mainDictionary[user].values():
        numberOfReviews += len(date)
print(numberOfReviews)




done
4803


In [ ]:
#10-05: Make user graph
import plotly.express as px
timeList = []
ylist = []
for item in freqDict['A2G7B7FKP2O2PU'].keys():
    timeList.append(item[0] + 12*(item[1]-2008))
    ylist.append(freqDict['A2G7B7FKP2O2PU'][item])
print(timeList)
print(ylist)

fig = px.scatter(x=timeList, y=ylist)

fig.update_layout(
    xaxis_title = "Month",
    yaxis_title = "Number of Reviews"
)
fig.show()
print("done")
#print(mainDictionary[next(iter(mainDictionary))])
print(dict(freqDict))

In [4]:
#Begin parcing through of user data to get averages for before, 
#peak month, 2 months after, extent after
from __future__ import division

import plotly.graph_objects as go

test = []
helpfulnessDict = {}
reviewDataDict = {}

for user in mainDictionary:
    sitems = sorted(mainDictionary[user].items(), key= lambda x: x[1][0][0], reverse=True)
    maxHelpfulness = sitems[0][1][0][0]
    maxHelpfulnessDate = sitems[0][0]
    helpfulnessDict[user] = [maxHelpfulness, maxHelpfulnessDate]
    

    
for user in freqDict:
    maxHelpfulness = helpfulnessDict[user][0]
    date = helpfulnessDict[user][1][0] + helpfulnessDict[user][1][1]*12
    reactionInterval = 2.0
    
    counterBefore = 0 #average monthly reviews before
    counterReaction = 0 #average monthly reviews 2 months aftter
    counterLongTerm = 0 #average monthly reviews from 2 months to rest of after
    
    reviewsBefore = 0
    reviewsDuring = 0
    reviewsReaction = 0
    reviewsLongTerm = 0
    
    for item in freqDict[user]:
        translatedDate = item[0] + item[1]*12
        
        if translatedDate < date and translatedDate: #>= date - reactionInterval:
            counterBefore += 1
            reviewsBefore += freqDict[user][item]
            
        elif translatedDate == date:
            reviewsDuring = freqDict[user][item]
            
        elif translatedDate > date and translatedDate <= date + reactionInterval:
            counterReaction += 1
            reviewsReaction += freqDict[user][item]
            
        elif translatedDate > reactionInterval:
            counterLongTerm += 1
            reviewsLongTerm += freqDict[user][item]
            
    if counterBefore == 0:
        avgBefore = 0.0
        
    else:
        avgBefore = reviewsBefore/counterBefore
        #avgBefore = reviewsBefore/reactionInterval
        
    if counterReaction == 0:
        avgReaction = 0.0
    else:
        avgReaction = reviewsReaction/counterReaction
        #avgReaction = reviewsReaction/reactionInterval
    
    if counterLongTerm == 0:
        avgLongTerm = 0.0
    else:
        avgLongTerm = reviewsLongTerm/counterLongTerm
    
    reviewDataDict[user] = [avgBefore, reviewsDuring, avgReaction, avgLongTerm]

    
#gives a dictionary where each user maps to an array :
#[average number of reviews per month BEFORE most helpful review, number of reviews 
#in the month with the most helpful review, number of reviews per month in the 
#deemed 'reaction' period, number of reviews per month after said reaction period]    

#print(reviewDataDict)

broadAvgBefore = 0.0
broadReviewsDuring = 0.0
broadAvgReaction = 0.0
broadAvgLongTerm = 0.0
broadChangeAvg = 0.0
broadChangeLongAvg = 0.0

numUsers = float(len(reviewDataDict.keys()))
changeNumUsers = numUsers

for user in reviewDataDict:
    #print(reviewDataDict[user])
    if reviewDataDict[user][0] > .000001 :
        broadAvgBefore += reviewDataDict[user][0]
        broadReviewsDuring += reviewDataDict[user][1]
        broadAvgReaction += reviewDataDict[user][2]
        broadAvgLongTerm += reviewDataDict[user][3]
    #print(reviewDataDict[user])
        #broadChangeAvg += (reviewDataDict[user][2] - reviewDataDict[user][0]) / reviewDataDict[user][0]
        #broadChangeLongAvg += (reviewDataDict[user][3] - reviewDataDict[user][0]) / reviewDataDict[user][0]
    else:
        changeNumUsers -= 1
print(numUsers)        
print(changeNumUsers)    


broadAvgBefore = broadAvgBefore/changeNumUsers
broadReviewsDuring = broadReviewsDuring/changeNumUsers
broadAvgReaction = broadAvgReaction/changeNumUsers
broadAvgLongTerm = broadAvgLongTerm/changeNumUsers
broadChangeAvg = broadChangeAvg/changeNumUsers
broadChangeLongAvg = broadChangeLongAvg/changeNumUsers
            
print("Before: " + str(broadAvgBefore))
print("During: " + str(broadReviewsDuring))
print("Reaction: " + str(broadAvgReaction))
print("Long: " + str(broadAvgLongTerm))
print("Percent change reaction: " + str((broadAvgReaction - broadAvgBefore)/broadAvgBefore))
print("Percent change long: " + str((broadAvgLongTerm - broadAvgBefore)/broadAvgBefore))
print("Percent change reaction to long: " + str((broadAvgLongTerm - broadAvgReaction)/broadAvgReaction))            
#print(str(item) + " " + str(freqDict[user][item]))


76.0
62.0
Before: 3.672598369372563
During: 6.580645161290323
Reaction: 4.467741935483871
Long: 3.8148628738739094
Percent change reaction: 0.2165070846685456
Percent change long: 0.03873674444985698
Percent change reaction to long: -0.1461317755228073


In [ ]:
#make a histogram dictionary of number of reviews by user

histogramDict = {}
for user in userDict:
    userCount = len(userDict[user])
    if userCount in histogramDict:
        histogramDict[userCount] += 1
    else:
        histogramDict[userCount] = 1
print(sorted(histogramDict.items()))
fig = px.bar(x = list(histogramDict.keys()), y = list(histogramDict.values()))
#fig.show()

#test that the number of reviews matches the number present in the data file
result = 0
for entry in histogramDict:
    result += entry * histogramDict[entry]
print (result == len(df.index))
print(max(histogramDict.keys()))

In [ ]:
import plotly
import plotly.express as px
from collections import OrderedDict
max = 0
maxItem = []
for item in userDict.values() :
    if len(item) > max :
        max = len(item)
        maxItem = item
timestamps = []
helpfulness = []
for review in maxItem :
    timestamps.append(review["Time"])
    if review["HelpfulnessDenominator"] == 0:
        helpfulness.append(0)
    else:
        helpfulness.append(review["HelpfulnessNumerator"])#/review["HelpfulnessDenominator"])
print(helpfulness)
fig = px.scatter(x=timestamps, y=helpfulness)
fig.show()

#sorted_d = OrderedDict(sorted(userDict.items(), key=lambda x: x[1]))
#print(sorted_d).head()

#factor = 2592000
#initialTime = maxItem[1]["Time"]
#freqList = [0,0,0,0,0]
#print(initialTime)
    

In [ ]:
import pandas as pd # import pandas library
import time
import plotly.express as px

df = pd.read_csv('Reviews.csv', low_memory=False)
print(df.columns)

byUser = df.groupby("UserId")


In [ ]:
#LIZZIE'S MESS AROUND CODE FOLLOWS

In [6]:
import pandas as pd # import pandas library
import time
import plotly.express as px

df = pd.read_csv('Reviews.csv', low_memory=False)# read the csv file into a pandas dataframe object
print(df.columns)

byUser = df.groupby("UserId")

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')


In [ ]:
#print(byUser.get_group("A3OXHLG6DIBRW8"))

print(byUser.get_group("A3OXHLG6DIBRW8")["HelpfulnessNumerator"])

bigUser = byUser.get_group("A3OXHLG6DIBRW8")

fig = px.scatter(x = (bigUser["HelpfulnessDenominator"] - bigUser["HelpfulnessNumerator"]), y = bigUser["HelpfulnessNumerator"])

fig.update_layout(
 xaxis_title = "Negative Reactions",
    yaxis_title = "Postive Reactions"
)

fig.show()

In [ ]:
bigUser = byUser.get_group("A3OXHLG6DIBRW8")

print(len(bigUser["HelpfulnessNumerator"]))

fig = px.scatter(x = bigUser["Time"],y = bigUser["HelpfulnessNumerator"])

fig.update_layout(
    xaxis_title = "Time",
    yaxis_title = "Helpfulness Numerator"

)
fig.show()

In [36]:
numReviews = []
avgHelpfulnessNumerator = []
print("here")

numHelpDict = {}

for id,data in byUser:
    rev = len(data["ProductId"])
    if rev in numHelpDict:
        for item in data['HelpfulnessNumerator']:
            numHelpDict[rev].append(item)
    else: 
        append = []
        for item in data["HelpfulnessNumerator"]:
            append.append(item)
            numHelpDict[rev] = append
    
    temp = 0
    numReviews.append(rev)
    for item in data["HelpfulnessNumerator"]:
        temp += item
    avg = temp/rev
    avgHelpfulnessNumerator.append(avg)
    
print("hi")

avgHelpfulnessByReviewX = []
helpfulnessValues = []
for number in numHelpDict:
    avgHelpfulnessByReviewX.append(number)
    helpfulnessValues.append((sum(numHelpDict[number]) / len(numHelpDict[number])))

    
    
fig = px.scatter(x = avgHelpfulnessByReviewX, y = helpfulnessValues)
fig.update_layout(
    xaxis_title = "Number of Reviews",
    yaxis_title = "Average Helpfulness Numerator"
)

fig.show()



here
hi


In [ ]:
#proportion of people that found the review helpful
numReviews = []
avgHelpfulness = []


for id,data in byUser: 
    if len(data["ProductId"]) != len(data["HelpfulnessDenominator"]):
        print(len(data["ProductId"]))
        print(len(data["HelpfulnessDenominator"]))
        print(" ")
    #rev = len(data["ProductId"])
    #temp = 0
    #numReviews.append(rev)
    #for i in range(rev) - 1:
     #   if data["HelpfulnessDenominator"][i] > 0:
      #      temp += data["HelpfulnessNumerator"]/float(data["HelpfulnessDenominator"])
    #avg = temp/rev
    #avgHelpfulness.append(avg)
    
fig = px.scatter(x = numReviews, y = avgHelpfulnessNumerator)
fig.update_layout(
    xaxis_title = "Number of Reviews",
    yaxis_title = "Average Proportion of Readers Finding Review Helpful"
)
#fig.show()

In [ ]:
numReviews = []
firstReviewHelpfulness = []

for id,data in byUser:
    rev = len(data["ProductId"])
    counter = 1
    numReviews.append(rev)
    for item in data["HelpfulnessNumerator"]:
        if counter > 0:
            firstReviewHelpfulness.append(item)
            counter = counter - 1 
    
print("hi")
fig = px.scatter(x = numReviews, y = firstReviewHelpfulness)
fig.update_layout(
    xaxis_title = "Number of Reviews",
    yaxis_title = "Helpfulness Numerator for First Review"
)

fig.show()

In [ ]:
bigUser = byUser.get_group("A3OXHLG6DIBRW8")

lengths = []
for item in bigUser["Text"]:
    lengths.append(len(item))
    
fig = px.scatter(x = bigUser["HelpfulnessNumerator"], y = lengths)
fig.update_layout(
    xaxis_title = "Helpfulness Numerator",
    yaxis_title = "Length of Review"
)

fig.show()